In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset

100% 23.1G/23.1G [07:43<00:00, 91.9MB/s]
100% 23.1G/23.1G [07:43<00:00, 53.4MB/s]


In [ ]:
import tensorflow as tf
tf.version.VERSION

'2.3.1'

In [ ]:
#!pip install --upgrade tensorflow==2.3.1 

In [ ]:
# now unzip the code
import zipfile
zip_ref = zipfile.ZipFile(r'/content/fashion-product-images-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

In [ ]:
model = ResNet50(weights = 'imagenet',include_top=False,input_shape=(224,224,3)) # we have to scalled down their shape coz, it is standdard size of the resnet50
# include_top= False | why? coz, we put our top layer or inshort we replce the top layer
model.trainable = False # we r not going to train model coz it alredy trained on imagent dataset so, we r just going to use

In [ ]:
# here we use our top layer
model = tensorflow.keras.Sequential([
        model, # it is all ResNet50 model without top layer
        GlobalAveragePooling2D() # here we added outr top layer
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
import numpy as np
from numpy.linalg import norm
import os 

In [ ]:
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224)) # load our image one by one with 224x224 resoulution
    img_array = image.img_to_array(img) # it convert that upper image into array format
    expanded_img_array = np.expand_dims(img_array, axis=0) # keras is going to read our image but in batchs so thats why we give iamge with batch numbers like (1,224,244,3) 1--> batch  | 224x224 --->resouliton | 3 is color com bination RGB
    preprocessed_img = preprocess_input(expanded_img_array) # it convert RGB to BGR coz, this model is based on imagnet dataset, so that, we have to convert our images also in that form which imagnet dataset have, and imagnet dataset's image in BGR from (not RGB) that why we convert our images also into RGB to BGR format
    result = model.predict(preprocessed_img).flatten() # we going to predict our model but in 1d array form that why we put flatten()  
    normalized_result = result / norm(result) # this process is give values between 0 to 1 like(0,0.0025,0.125,0.112,0.1574585,1))
    # norm() function is used to convert for normlize the values
    return normalized_result

In [ ]:
filenames = [] # this list have to store full path of our images with the help of below loop
for file in os.listdir('/content/fashion-dataset/images'):
  filenames.append(os.path.join('/content/fashion-dataset/images',file))

In [ ]:
from tqdm import tqdm # for loop tracking in percentage form

In [ ]:
features_list = [] # this list have to store our images some values with is called feature extracted values , thats the value is classifed our images like( shirt image, bat image, jacket image etc)
for file in tqdm(filenames):
  features_list.append(extract_features(file,model))

100%|██████████| 44441/44441 [3:12:50<00:00,  3.84it/s]


In [ ]:
import pickle
pickle.dump(features_list ,open('embeddings1.pkl','wb'))
pickle.dump(filenames ,open('filenames1.pkl','wb'))